In [5]:
"""ДЕТЕКЦІЯ АВТОМОБІЛІВ НА КОЖНОМУ N-НОМУ КАДРІ ВІДЕОПОТОКУ"""

import os
import cv2
import numpy as np
import pywt
from ultralytics import YOLO

def apply_wavelet_centered(img, wavelet='db6', lambda_=0.5):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    coeffs2 = pywt.dwt2(gray, wavelet)
    _, (H, V, D) = coeffs2
    W = np.sqrt(H ** 2 + V ** 2 + D ** 2)
    W = cv2.resize(W, (img.shape[1], img.shape[0]))
    W_centered = W - np.mean(W)
    img_float = img.astype(np.float32)
    W3 = np.stack([W_centered] * 3, axis=-1)
    result = img_float + lambda_ * W3
    return np.clip(result, 0, 255).astype(np.uint8)

def extract_and_detect_frames(model_path, video_path, output_folder, frame_step=30):
    os.makedirs(output_folder, exist_ok=True)
    model = YOLO(model_path)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_step == 0:
            wavelet_frame = apply_wavelet_centered(frame.copy(), lambda_=0.5)
            results = model.predict(wavelet_frame, conf=0.3, verbose=False)[0]

            for box in results.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

            save_path = os.path.join(output_folder, f"frame_{saved_count:04d}.jpg")
            cv2.imwrite(save_path, frame)
            saved_count += 1

        frame_count += 1
    cap.release()
    print(f"Оброблено {saved_count} кадрів")

if __name__ == '__main__':
    extract_and_detect_frames(
        model_path='C:/Users/nasti/PyCharmProjects/CourseProject/model.pt',
        video_path='C:/Users/nasti/PyCharmProjects/CourseProject/orig_data/video3.mp4',
        output_folder='C:/Users/nasti/PyCharmProjects/CourseProject/res_from_video',
        frame_step=32  # кожен n-й кадр (орієнтовно 1 секунда при 30 FPS)
    )


Оброблено 35 кадрів
